<a href="https://colab.research.google.com/github/JaledMC/Learning-Artificial-Vision/blob/master/movement_face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video features detection

In [0]:
# Import the libraries

import cv2
import numpy as np
import matplotlib.pyplot as plt

In [0]:
import requests

response = requests.get('https://raw.githubusercontent.com/JaledMC/Learning-Artificial-Vision/master/images/car.mp4')
with open('car.mp4', 'wb') as f:
    f.write(response.content)

## Movement object extraction

Movement object segmentation over a static background is a clasical problem. At first, it can be enought subtracting two sequential frames, changes in ambient light or shadows can produce noises. 

In [0]:
cap = cv2.VideoCapture('car.mp4')
# MOG2 substactor
fgbg = cv2.createBackgroundSubtractorMOG2()

while(1):
    ret, frame = cap.read()
    # obtain the foreground mask
    fgmask = fgbg.apply(frame)
    
    cv2.imshow('fgmask',frame)
    cv2.imshow('frame',fgmask)   
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
cap.release()
cv2.destroyAllWindows()

## Viola-James Haar cascade

Viola-James Haar cascade is used typically for face detection. One good thing of Viola-Jones algorithm is its small size: xml files of trained features barely wheights a couple of MB. Thats means that with a normal 4GB RAM we can clasify thousands of objects. [We can use xml for other people](# multiple cascades: https://github.com/Itseez/opencv/tree/master/data/haarcascades), or [train the algorithm to detect our objects](https://pythonprogramming.net/haar-cascade-object-detection-python-opencv-tutorial/), but their training are slow.

Haar cascades are not too fast, because they have to run over all the images multiple times, with different kernels size. To speed up the program, for complex objects like cars or faces, it's better to use hierarchical segmentation: first detect the face, and over the ROI of the face, detect the eyes, mounth...

Training a Haar Cascade it's time consuming. We can find trained examples [here](#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml).

First, download the xml file and create the cascade classifier.

In [0]:
# Import Haar cascades database for face and eyes, and instace the detector objects
import requests
# Create the face detector
response = requests.get('https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml')
with open('haarcascade_frontalface_default.xml', 'wb') as f:
    f.write(response.content)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Create the eye detector
response = requests.get('https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml')
with open('haarcascade_frontalface_default.xml', 'wb') as f:
    f.write(response.content)
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

In [0]:
cap = cv2.VideoCapture(1)

while True:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect the position and area of faces in the image: image, scaling step, pixel number for the area
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    # plot the square rectangle and select the ROI
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = img[y:y + h, x:x + w]
        # detect the position and area of eyes on the roi
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex + ew, ey + eh), (0, 255, 0), 2)

    cv2.imshow('img', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

We can detect eyes too.

In [0]:
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

cap = cv2.VideoCapture(1)

while True:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect the position and area of eyes in the image: image, scaling step, pixel number for the area
    eyes = eye_cascade.detectMultiScale(gray, 1.3, 5)
    # plot the square rectangle 
    for (x,y,w,h) in eyes:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

    cv2.imshow('img', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()